In [1]:
# import all modules needed
import pandas as pd
import numpy as np

# import csv data
imported_data1 = pd.read_csv('drugsComTrain_raw.csv')
imported_data1.drop(columns={"date","usefulCount"}, inplace=True)


In [2]:
# make condition missing/nan for all rows in which variable condition contains 
# the following # unwanted text: "</span> users found this comment helpful."
temp1 = imported_data1[imported_data1['condition'].str.contains("/span> users found this comment helpful.", na=False)]
temp1.loc[:, 'condition'] = np.nan
temp2 = imported_data1[~imported_data1['condition'].str.contains("/span> users found this comment helpful.", na=False)]
imported_data2 = pd.concat([temp1, temp2], axis=0)


In [3]:
# find all rows with condition variable missing/nan, then fill in from 
# other rows containing the same drugName

# condition is missing
data_blank_condition = imported_data2[pd.isna(imported_data2['condition'])]

# condition is not missing
data_non_blank_condition = imported_data2[~pd.isna(imported_data2['condition'])]

# get a dataset of unique drugName with corresponding condition
unique_drug_condition = data_non_blank_condition[['drugName', 'condition']].groupby('drugName').first().reset_index()
unique_drug_condition.rename(columns={"condition":"new_condition"}, inplace=True)

# merge with condition missing dataset to get condition for each drug
merged_data = pd.merge(data_blank_condition, unique_drug_condition, on='drugName', how='left')
merged_data['condition'] = merged_data['condition'].fillna(merged_data['new_condition'])
merged_data.drop(columns='new_condition', inplace=True)

# keep only rows with condition not missing
imported_data3 = pd.concat([data_non_blank_condition, merged_data], axis=0)
imported_data3 = imported_data3[~pd.isna(imported_data3['condition'])]

# ensure condition column is string type
imported_data3.loc[:, 'condition'] = imported_data3['condition'].astype(str)


In [4]:
# sort data by condition then drug so that numeric variables follow this
imported_data3.sort_values(by=['condition', 'drugName'], inplace=True)
imported_data3.reset_index(drop=True, inplace=True)

# get a sorted list of unique drug names
unique_drug = imported_data3['drugName'].unique()

# get a sorted list of unique conditions
unique_cond = imported_data3['condition'].unique()

# create dictionaries mapping each unique drug name and condition to a unique number
drug_name_mapping = {drug_name: i for i, drug_name in enumerate(unique_drug)}
condition_mapping = {condition: i for i, condition in enumerate(unique_cond)}

# map these numbers to drugName and condition columns
imported_data3['drugName_num'] = imported_data3['drugName'].map(drug_name_mapping)
imported_data3['condition_num'] = imported_data3['condition'].map(condition_mapping)


In [5]:
imported_data3


,uniqueID,drugName,condition,review,rating,drugName_num,condition_num
0,88437,Adderall,ADHD,"""Helps me keep my attention in class, but see...",6,0,0
1,88396,Adderall,ADHD,"""This medicine is amazing. I used to ALWAYS be...",10,0,0
2,88409,Adderall,ADHD,"""Hyper focused, dry mouth. Both this and Strat...",4,0,0
3,88532,Adderall,ADHD,"""I have ADHD and have been taking Adderall for...",8,0,0
4,88405,Adderall,ADHD,"""Keeps you focused. However when it wears off ...",9,0,0
...,...,...,...,...,...,...,...
161268,87551,Naproxen,zen Shoulde,"""Very little relief. I finished PT and after ...",2,488,810
161269,87689,Naproxen,zen Shoulde,"""One every 8 hours dulls the pain but sufficie...",5,488,810
161270,87803,Naproxen,zen Shoulde,"""I took three pills over 36 hours and had a se...",1,488,810
161271,111774,Relafen,zen Shoulde,"""I&#039;m probably the only one I know taking ...",7,750,810
